<a href="https://colab.research.google.com/github/hariharanrl/NLP/blob/master/NLP_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Google Drive

In [0]:
import pandas as pd
import os
os.chdir('/content/drive/My Drive')

#Spacy (Not needed)

In [0]:
#import spacy
#nlp=spacy.load('en_core_web_sm')#lg large library #sm small library

In [0]:
#stopwords=nlp.Defaults.stop_words

In [53]:
#len(stopwords)

326

In [51]:
#nlp.vocab['@user'].is_stop

False

In [0]:
#to add a stop word
#nlp.Defaults.stop_words.add('@user') 
#nlp.vocab['@user'].is_stop=True

#Download the Vader Class from NLTK

In [50]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

##Load the data

In [0]:
data=pd.read_csv('task_b_distant.tsv',sep='\t')

##Install and Import Tweet Preprocessor

In [112]:
!pip install tweet-preprocessor

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=084ddabd2abc46a1d62968f24a8630199768a42c66c361e9bb35f2f9350d0315
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [0]:
import preprocessor as p

##Clean the tweet in the imported data using preprocessor

In [114]:
p.clean(data.loc[0]['text'])

'His ass need to stay up'

In [115]:
data['cleaned_tweet']=data['text'].apply(lambda text:p.clean(text))
data.head()

,id,text,average,std,scores,label,class,cleaned_tweet
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,"{'neg': 0.333, 'neu': 0.667, 'pos': 0.0, 'comp...",-0.5423,off,His ass need to stay up
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,"{'neg': 0.456, 'neu': 0.544, 'pos': 0.0, 'comp...",-0.7713,off,Trump is a fucking idiot his dementia is getti...
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,"{'neg': 0.447, 'neu': 0.288, 'pos': 0.265, 'co...",-0.6988,off,HELL YES! His grinned and thumbs up are disgus...
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,"{'neg': 0.218, 'neu': 0.559, 'pos': 0.223, 'co...",0.0258,not,Can't wait to see the shit show his death will...
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,"{'neg': 0.355, 'neu': 0.645, 'pos': 0.0, 'comp...",-0.7650,off,This guys is dumb check his latest tweets he i...


##Use the vader class to calculate the polarity scores for the cleaned tweets

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


In [13]:
sid.polarity_scores(data.loc[15]['text'])

{'compound': -0.802, 'neg': 0.31, 'neu': 0.69, 'pos': 0.0}

In [116]:
data['scores']=data['cleaned_tweet'].apply(lambda text:sid.polarity_scores(text))
data.head()

,id,text,average,std,scores,label,class,cleaned_tweet
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp...",-0.5423,off,His ass need to stay up
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,"{'neg': 0.489, 'neu': 0.511, 'pos': 0.0, 'comp...",-0.7713,off,Trump is a fucking idiot his dementia is getti...
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,"{'neg': 0.469, 'neu': 0.252, 'pos': 0.279, 'co...",-0.6988,off,HELL YES! His grinned and thumbs up are disgus...
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,"{'neg': 0.231, 'neu': 0.533, 'pos': 0.237, 'co...",0.0258,not,Can't wait to see the shit show his death will...
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,"{'neg': 0.423, 'neu': 0.577, 'pos': 0.0, 'comp...",-0.7650,off,This guys is dumb check his latest tweets he i...


##Label the tweet using the compound score

In [117]:
data['label']=data['scores'].apply(lambda d:d['compound'])
data.head()

,id,text,average,std,scores,label,class,cleaned_tweet
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp...",-0.5423,off,His ass need to stay up
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,"{'neg': 0.489, 'neu': 0.511, 'pos': 0.0, 'comp...",-0.7713,off,Trump is a fucking idiot his dementia is getti...
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,"{'neg': 0.469, 'neu': 0.252, 'pos': 0.279, 'co...",-0.6988,off,HELL YES! His grinned and thumbs up are disgus...
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,"{'neg': 0.231, 'neu': 0.533, 'pos': 0.237, 'co...",0.0258,not,Can't wait to see the shit show his death will...
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,"{'neg': 0.423, 'neu': 0.577, 'pos': 0.0, 'comp...",-0.7650,off,This guys is dumb check his latest tweets he i...


In [118]:
data['class']=data['label'].apply(lambda c:'not' if c>=0 else 'off')
data.head()

,id,text,average,std,scores,label,class,cleaned_tweet
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp...",-0.5423,off,His ass need to stay up
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,"{'neg': 0.489, 'neu': 0.511, 'pos': 0.0, 'comp...",-0.7713,off,Trump is a fucking idiot his dementia is getti...
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,"{'neg': 0.469, 'neu': 0.252, 'pos': 0.279, 'co...",-0.6988,off,HELL YES! His grinned and thumbs up are disgus...
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,"{'neg': 0.231, 'neu': 0.533, 'pos': 0.237, 'co...",0.0258,not,Can't wait to see the shit show his death will...
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,"{'neg': 0.423, 'neu': 0.577, 'pos': 0.0, 'comp...",-0.7650,off,This guys is dumb check his latest tweets he i...


##Check for any Null texts

In [20]:
data.isnull().sum()

id         0
text       0
average    0
std        0
scores     0
label      0
class      0
dtype: int64

##Check for blanks

In [0]:
blanks=[]
for i,j,text,avg,std in data.itertuples():
  if type(text)==str:
    if text.isspace():
      blanks.append(i)
print(len(blanks))

In [0]:
#data['label']="not"
#data.loc[data['average']>0.20,'label']="off"


In [119]:
data['class'].value_counts()

off    161621
not     27106
Name: class, dtype: int64

#Training and test split 

In [0]:
from sklearn.model_selection import train_test_split
X=data['cleaned_tweet']
y=data['class']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

##Building a Naive bayes and Linear SVM model 

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

#text_cls_nb=Pipeline([('tfidf',TfidfVectorizer(stop_words='english')),('clf',MultinomialNB()),])
text_cls_svm=Pipeline([('tfidf',TfidfVectorizer(stop_words='english')),('clf',LinearSVC()),])

##Fit the model with the train data

In [122]:
#text_cls_nb.fit(X_train,y_train)
text_cls_svm.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
          

##Prediction, Confusion Matrix and Classification Report

In [0]:
#prediction=text_cls_nb.predict(X_test)
prediction=text_cls_svm.predict(X_test)

In [124]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,prediction))

[[ 2306  3002]
 [  862 31576]]


In [125]:
print(metrics.classification_report(y_test,prediction))

              precision    recall  f1-score   support

         not       0.73      0.43      0.54      5308
         off       0.91      0.97      0.94     32438

    accuracy                           0.90     37746
   macro avg       0.82      0.70      0.74     37746
weighted avg       0.89      0.90      0.89     37746



In [126]:
print(metrics.accuracy_score(y_test,prediction))

0.8976315371165157


In [127]:
print(text_cls_svm.predict(['I love him , he is so awesome']))

['not']


In [128]:
print(text_cls_svm.predict(['What the fuck?']))

['off']
